<a href="https://colab.research.google.com/github/jessica-aaao/ChordsExtractor/blob/main/ChordExtractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [22]:
!pip install -q -U "yt-dlp[default]"
!pip install -q -U openai-whisper
!pip install -q -U demucs

In [23]:
import requests
import whisper
import json
import pandas as pd
import os
import re
import unicodedata

In [24]:
from google.colab import drive
from IPython.display import display
from bs4 import BeautifulSoup

In [25]:
!git clone https://github.com/filipecalegario/ISMIR2019-Large-Vocabulary-Chord-Recognition.git

%cd ISMIR2019-Large-Vocabulary-Chord-Recognition
!pip install -q -r requirements.txt
%cd ..

fatal: destination path 'ISMIR2019-Large-Vocabulary-Chord-Recognition' already exists and is not an empty directory.
/content/ISMIR2019-Large-Vocabulary-Chord-Recognition
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
/content


In [26]:
!git clone  https://github.com/mikezzb/lyrics-sync.git

!wget -c https://repo.continuum.io/archive/Anaconda3-2024.10-1-Linux-x86_64.sh
!chmod +x Anaconda3-2024.10-1-Linux-x86_64.sh
!bash ./Anaconda3-2024.10-1-Linux-x86_64.sh -b -f -p /usr/local

fatal: destination path 'lyrics-sync' already exists and is not an empty directory.
--2025-03-08 01:58:57--  https://repo.continuum.io/archive/Anaconda3-2024.10-1-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.176.84, 104.18.177.84, 2606:4700::6812:b154, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.176.84|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/archive/Anaconda3-2024.10-1-Linux-x86_64.sh [following]
--2025-03-08 01:58:57--  https://repo.anaconda.com/archive/Anaconda3-2024.10-1-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.32.241, 104.16.191.158, 2606:4700::6810:bf9e, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.32.241|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

PREFIX=/usr/local
Unpacking payload ...

Installing b

In [9]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!apt-get install ffmpeg
!apt-get install espeak

!git clone https://github.com/readbeyond/aeneas.git
%cd aeneas
!pip install -r requirements.txt
!python check_dependencies.py
%cd ..

# Common

In [10]:
class SongUrls:
    def __init__(self, name, audio, lyrics, chords):
        self.name = name
        self.audio = audio
        self.lyrics = lyrics
        self.chords = chords

    def get_name(self):
        return self.name

    def get_audio_url(self):
        return self.audio

    def get_lyrics_url(self):
        return self.lyrics

    def get_chords_url(self):
        return self.chords

In [11]:
def get_urls():
    print(f'Fetching urls...\n\n')

    file_path = '/content/drive/My Drive/TCC/CodeData/songs.csv'
    songs = pd.read_csv(file_path)

    print(f'Urls fetched!\n\n')

    return songs

In [28]:
def get_songs_from_csv():
    songs = get_urls()

    song_urls = []

    print(f'Creating SongUrls...\n\n')

    for index, row in songs.iterrows():
        special_case = row['Special Case']
        if special_case:
            continue

        song_name = slugify(row['Song Name'])
        audio_url = row['Audio URL']
        lyrics_url = row['Lyrics URL']
        chords_url = row['Chords URL']

        song_urls.append(SongUrls(song_name, audio_url, lyrics_url, chords_url))

    print(f'SongUrls created!\n\n')

    return song_urls


In [27]:
def slugify(raw_song_name):
    song_name = raw_song_name.lower()

    song_name = unicodedata.normalize('NFKD', song_name)
    song_name = song_name.encode('ascii', 'ignore').decode('ascii')

    song_name = re.sub(r'[^a-z0-9]+', '_', song_name)

    song_name = song_name.strip('_')

    return song_name

# Sound

In [13]:
def extract_voice_from_audio(audio_path, song_name):
    print(f'Extracting vocals from {audio_path}...\n\n')

    !demucs --two-stems=vocals {audio_path} -o vocals/

    return f"/content/vocals/htdemucs/{song_name}/vocals.wav"


In [31]:
def extract_audio_from_video(youtube_url, song_name):
    cookies_path = '/content/cookies.txt'
    output_path = f"/content/audios/{song_name}"

    !yt-dlp {youtube_url} --audio-format "wav" --cookies {cookies_path} -x -o {output_path}  -q

    print(f'Audio saved as {output_path}.wav!\n\n')

    return f"{output_path}.wav"

In [15]:
def extract_sound_recording(youtube_url, song_name):
    output_path = extract_audio_from_video(youtube_url, song_name)
    # vocals_path = extract_voice_from_audio(f"{output_path}.wav", song_name)

    print(f'Audio saved as {output_path}!\n\n')

    return output_path

# Lyrics

## Lyrics Extraction from Webpage

In [16]:
def save_to_file(data, song_name):
    folder_path = "/content/lyrics"
    path = f"{folder_path}/{song_name}.txt"

    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    with open(path, 'w') as file:
        file.write(data)

    print(f'Saved as {path}')

    return path

In [17]:
def extract_lyrics_from_html(html):
    """Extrai a letra da página HTML fornecida"""

    print(f'Fetching lyrics...!\n\n')

    lyricsTag = html.find('div', class_='lyric-original')
    lyrics = ""

    for p in lyricsTag.find_all('p'):
        for br in p.find_all('br'):
            br.replace_with('\n')
        lyrics += p.get_text() + "\n"

    print(f'Lyrics fetched!\n\n')

    return lyrics


In [18]:
def get_lyrics_from_webpage(lyric_url, song_name):
    """Obtém a página web e extrai a letra"""

    print(f'Fetching webpage {lyric_url}...\n\n')
    response = requests.get(lyric_url)

    if response.status_code == 200:
        print(f'Webpage fetched!\n\n')
        htmlContent = BeautifulSoup(response.content, 'html.parser')

        lyrics = extract_lyrics_from_html(htmlContent)
        lyrics_path = save_to_file(lyrics, song_name)

        return lyrics_path
    else:
        print(f"Failed to fetch the webpage. Status code: {response.status_code}\n\n")

## Lyrics Sync to Audio

### Transcription + Manual Attempt

In [ ]:
def adjust_lyrics(transcribed_segments, comparison_lyrics):
    """Compara a letra transcrita e a letra extraída da Web, corrigindo a letra transcrita"""

    print(f"Adjusting lyrics...\n\n")

    for index, segment in enumerate(transcribed_segments):
        try:
            line = comparison_lyrics[index]

            if segment['text'].lower() != line.lower():
                segment['text'] = line

            words = line.split()
            # Testar com músicas não "especiais", mais enxutas
            if len(words) != len(segment['words']):
                print(f"Word count mismatch: {len(words)} != {len(segment['words'])}")
                continue

            for i, word in enumerate(words):
                try:
                    transcribed_word = segment['words'][i]['word']

                    if transcribed_word.lower().strip() != word.lower():
                        print(f"Word mismatch: {transcribed_word} != {word}")

                        segment['words'][i]['word'] = word
                except:
                    print(f"Word not found: {word}")
                    continue
        except:
            print(f"Line not found: {segment['text']}")
            continue

    print(f"Lyrics adjusted:\n{json.dumps(transcribed_lyrics, indent=2)}\n\n")

    return transcribed_lyrics

In [ ]:
def transcribe_audio(audio_path):
    """Transcreve o áudio usando o modelo Whisper"""

    print(f"Transcribing audio...\n\n")

    model = whisper.load_model("turbo", device='cuda')
    result = model.transcribe(audio_path, word_timestamps=True)

    print(f"Audio Transcribed!\n\n")

    return result['segments']


### Using Lyrics-Sync

In [35]:
def create_output_folder():
    output_folder = "/content/lyrics-sync/output"
    vocals_folder = output_folder + "/vocals"
    words_folder = output_folder + "/words"
    lrc_folder = output_folder + "/lrc"

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    if not os.path.exists(vocals_folder):
        os.makedirs(vocals_folder)

    if not os.path.exists(words_folder):
        os.makedirs(words_folder)

    if not os.path.exists(lrc_folder):
        os.makedirs(lrc_folder)

In [33]:
def get_timestamps(audio_path, lyrics_path, song_name):
    %cd /content/lyrics-sync/
    !conda env update -f environment.yml
    !source activate lsync

    from lsync import LyricsSync

    lsync = LyricsSync()
    words, lrc = lsync.sync(audio_path, lyrics_path)
    print(words)
    print(lrc)

    return f"/content/lyrics-sync/output/{song_name}.lrc"
    %cd ..

In [34]:
def get_synced_lyrics(lyric_url, audio_path, song_name):
    """Obtém a letra sincronizada com o áudio e corrigida"""

    print(f"Getting synced lyrics...\n\n")

    # transcribed_lyrics = transcribe_audio(audio_path)
    lyrics_path = get_lyrics_from_webpage(lyric_url, song_name)
    timestamps_path = get_timestamps(audio_path, lyrics_path, song_name)

    print(f"Synced lyrics ready!\n\n")

    # return (transcribed_lyrics, comparison_lyrics)
    return timestamps_path

# Chords

## Chords Extraction from Audio

In [ ]:
def extract_cipher_from(song_path, song_name):
    """Extrai a cifra, com o timestamp, a partir da música"""
    output_path = f"/content/chords/{song_name}.lab"
    !python chord_recognition.py {song_path} {output_path}

    return output_path

## Chords Sync to Lyrics

In [ ]:
def format_song_with_chords(transcribed_lyrics_path, chords_path):
    """
    Formata a LETRA_TRANSCRITA com acordes no formato tradicional de cifras.
    """
    # Ler LETRA_TRANSCRITA do JSON
    with open(transcribed_lyrics_path, 'r', encoding='utf-8') as file:
        transcribed_lyrics = json.load(file)

    # Ler ACORDES do arquivo LAB
    chords = []
    with open(chords_path, 'r', encoding='utf-8') as file:
        for line in file:
            start, end, chord = line.strip().split('\t')
            chords.append((float(start), float(end), chord))

    # Inserir acordes nos timestamps
    lyrics_with_chords = overlay_chords_on_transcribed(transcribed_lyrics, chords)
    # print(lyrics_with_chords)

    # Formatar saída com acordes e palavras
    formatted_output = format_transcribed_with_chords(lyrics_with_chords)

    return formatted_output

In [ ]:
def overlay_chords_on_transcribed(transcribed_lyrics, chords):
    """
    Associa acordes às palavras da LETRA_TRANSCRITA com base nos tempos de ACORDES.
    """
    result = []
    for segment in transcribed_lyrics.get("segments", []):
        for word_info in segment.get("words", []):
            start = word_info.get("start", None)
            end = word_info.get("end", None)
            word = word_info.get("word", "")
            chord = None

            if start is not None and end is not None:
                # Encontrar acorde correspondente ao intervalo
                for chord_start, chord_end, chord_name in chords:
                    if chord_start <= start < chord_end:
                        chord = chord_name
                        break

            result.append({
                "word": word,
                "start": start,
                "end": end,
                "chord": chord
            })
    return result

In [ ]:
def format_transcribed_with_chords(lyrics_with_chords):
    """
    Gera o formato de saída com acordes acima das palavras.
    """
    formatted_output = []
    chord_line = []
    lyrics_line = []

    for word_info in lyrics_with_chords:
        word = word_info["word"]
        chord = word_info["chord"]

        if chord:
            chord_line.append(f"{chord}")
        else:
            chord_line.append(" " * len(word))

        lyrics_line.append(word)

        # Adiciona quebra de linha para segmentos
        if word.endswith((".", ",", "?")):
            formatted_output.append(" ".join(chord_line).rstrip())
            formatted_output.append(" ".join(lyrics_line).rstrip())
            formatted_output.append("")
            chord_line = []
            lyrics_line = []

    # Adiciona o restante, se existir
    if chord_line or lyrics_line:
        formatted_output.append(" ".join(chord_line).rstrip())
        formatted_output.append(" ".join(lyrics_line).rstrip())

    return "\n".join(formatted_output)

In [ ]:
def simplify_chords_with_lyrics(chords_line, lyrics_line):
    """
    Simplifica uma linha de acordes e palavras associadas.
    Remove redundâncias e organiza a saída.
    """
    chords = chords_line.split()
    lyrics = lyrics_line.split()

    simplified_chords = []
    simplified_lyrics = []

    # Último acorde usado
    last_chord = None

    for chord, word in zip(chords, lyrics):
        if chord != last_chord:
            simplified_chords.append(chord)
            simplified_lyrics.append(word)
            last_chord = chord
        else:
            simplified_chords.append("")  # Espaço vazio na linha de acordes
            simplified_lyrics.append(word)

    # Retorna linhas formatadas
    return " ".join(simplified_chords).rstrip(), " ".join(simplified_lyrics).rstrip()

In [ ]:
def format_clean_chords_and_lyrics(raw_chords_lyrics):
    """
    Formata o texto contendo acordes redundantes e letras em um formato limpo e tradicional.
    """
    formatted_output = []
    lines = raw_chords_lyrics.strip().split("\n")

    # Processa par de linhas (chord, lyrics)
    for i in range(0, len(lines), 2):
        if i + 1 < len(lines):  # Certifique-se de que há uma linha de letras
            chords_line = lines[i]
            lyrics_line = lines[i + 1]

            # Simplificar a linha
            simplified_chords, simplified_lyrics = simplify_chords_with_lyrics(chords_line, lyrics_line)

            # Adicionar ao resultado
            formatted_output.append(simplified_chords)
            formatted_output.append(simplified_lyrics)
            formatted_output.append("")  # Linha em branco para separar blocos

    return "\n".join(formatted_output).strip()



---



# MAIN

## Inputs

In [ ]:
song_url = input("Enter youtube url: ")
# lyric_url= input("Enter lyric url: ")



KeyboardInterrupt: Interrupted by user

In [ ]:
extract_sound_recording(song_url, "nome")

## From CSV

In [36]:
# Extrai dados do csv e converte pra [SongUrls]
songs = get_songs_from_csv()

song = songs[0]
song_name = song.get_name().replace(" ", "_")
print(f"Generating \'{song_name}\' chord sheet...")

Fetching urls...


Urls fetched!


Creating SongUrls...


SongUrls created!


Generating 'que_pais_e_este' chord sheet...


In [37]:
# Baixa músicas do youtube
song_path = extract_sound_recording(song.get_audio_url(), song_name)

Audio saved as /content/audios/que_pais_e_este.wav.wav!


Audio saved as /content/audios/que_pais_e_este.wav!




In [39]:
# Extrai letras com timestamp
create_output_folder()
lyrics_timestamped_path = get_synced_lyrics(song.get_lyrics_url(), song_path, song_name)

Getting synced lyrics...


Fetching webpage https://www.letras.mus.br/legiao-urbana/46973/...


Webpage fetched!


Fetching lyrics...!


Lyrics fetched!


Saved as /content/lyrics/que_pais_e_este.txt
/content/lyrics-sync
/usr/local/lib/python3.12/argparse.py:2006: FutureWarning: `remote_definition` is deprecated and will be removed in 25.9. Use `conda env create --file=URL` instead.
  action(self, namespace, argument_values, option_string)
Channels:
 - pytorch
 - nvidia
 - defaults
 - conda-forge
Platform: linux-64
Solving environment: | / - done
Installing pip dependencies: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - 

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[<00:40.75> Nas, <00:40.93> favelas,, <00:42.43> no, <00:42.61> Senado, <00:44.93> Sujeira, <00:45.28> pra, <00:45.50> todo, <00:45.93> lado, <00:47.16> Ninguém, <00:47.64> respeita, <00:48.06> a, <00:48.17> Constituição, <00:50.89> Mas, <00:51.10> todos, <00:51.57> acreditam, <00:52.38> no, <00:52.50> futuro, <00:53.16> da, <00:53.39> nação, <00:57.24> Que, <00:57.43> país, <00:58.06> é, <00:58.17> esse?, <01:00.52> Que, <01:00.70> país, <01:01.32> é, <01:01.48> esse?, <01:03.82> Que, <01:04.01> país, <01:04.68> é, <01:04.82> esse?, <01:20.32> No, <01:20.50> Amazonas,, <01:21.96> no, <01:22.06> Araguaia-ia-ia, <01:23.65> Na, <01:23.82> Baixada, <01:24.42> Fluminense, <01:26.87> Mato, <01:27.28> Grosso,, <01:28.40> Minas, <01:28.78> Gerais, <01:29.98> E, <01:30.14> no, <01:30.34> Nordeste, <01:31.20> tudo, <01:31.48> em, <01:31.68> paz, <01:33.29> Na, <01:33.50> morte,, <01:34.01> eu, <01:34.32> descanso, <01:36.78> Mas, <01:36.98> o, <01:37.14> sangue, <01:37.46> anda, <01:37.82> solt